In [1]:
%matplotlib inline
import numpy  as np 
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
from jupyterthemes import jtplot
jtplot.style()

In [59]:
class Disease():
    def __init__(self, idx=-1, name='', mx=[], link=[]):
        self.idx  = idx   # disease number
        self.name = name  # disease name
        self.mx   = mx    # assoc. manifestations
        self.link = link  # linked diseases
        
    # get functions
    def get_idx(self):   return self.idx
    def get_name(self):  return self.name
    def get_mxs(self):   return self.mx
    def get_links(self): return self.link
    
    # add functions
    def add_mx(self, new_mx):
        self.mx.append(new_mx)
        
    def add_link(self, new_link):
        self.link.append(new_link)
        
    # is_in functions
    def has_manifestation(self, m):
        return True if m in self.mx else False
    
    def has_link(self, l):
        return True if l in self.link else False 

In [54]:
class Manifestation():
    def __init__(self, idx=-1, name='', es=-1, fq=-1, im=-1):
        self.idx  = idx   # mx number
        self.name = name  # mx name
        self.es   = es    # evoking strength
        self.fq   = fq    # frequency
        self.im   = im    # import
        
    def __eq__(self, other):
        return self.idx == other.idx
    
    def __ne__(self, other):
        return not self == other
    
    def __str__(self):
        return '{}: {} | es = {}, fq = {}, im = {}'.format(self.idx, self.name,
                                                             self.es, self.fq, self.im)
        
    # nonlinear transformations
    def trans_es(self):
        trans = {0: 1, 1: 4, 2: 10, 3: 20, 4: 40, 5: 80}
        return trans[self.es]

    def trans_fq(self):
        trans = {1: -1, 2: -4, 3: -7, 4: -15, 5: -30}
        return trans[self.fq]

    def trans_im(self):
        trans = {1: -2, 2: -6, 3: -10, 4: -20, 5: -40}
        return trans[self.im]
        
    # get functions
    def get_idx(self):  return self.idx
    def get_name(self): return self.name
    def get_es(self):   return trans_es(self.es)
    def get_fq(self):   return trans_fq(self.fq)
    def get_im(self):   return trans_im(self.im)
    
    # set functions
    def set_es(self, new_es):
        self.es = new_es
    def set_fq(self, new_fq):
        self.fq = new_fq
    def set_im(self, new_im):
        self.im = new_im

In [5]:
class Patient():
    def __init__(self, pos=[], neg=[]):
        self.positive = pos  # + findings
        self.negative = neg  # - findings
        
    # get functions
    def get_positives(self): return self.positive
    def get_negatives(self): return self.negative
    
    # add functions
    def add_positive(self, new_pos):
        self.positive.append(new_pos)
        
    def add_negative(self, new_neg):
        self.negative.append(new_neg)

In [25]:
def parse_disease(line):
    return line[1], ' '.join(line[2:])

def parse_manifestation(line):
    return line[2], ' '.join(line[3:]), line[1][0], line[1][1]

def parse_link(line):
    return line[3], ' '.join(line[4:])

def parse_even(line):
    return line[1], ' '.join(line[2:])

def parse_odd(line):
    return line[0], line[2]

In [56]:
FINDINGS = {}
DISEASES = {}

In [58]:
# build master manifestations list from findings file (#, NAME, IM)
with open('./data/findings.txt', 'r') as fd_file:
    for i, line in enumerate(fd_file):
        line = line.strip('\n').split()
        if i % 2:
            idx, imp = parse_odd(line)
            FINDINGS[idx].set_im(imp)
        else:
            idx, name = parse_even(line)
            FINDINGS[idx] = Manifestation(idx, name)

In [60]:
# build set of diseases (#, NAME, MX, LINKS) and update manifestations (ES and FQ)
with open('./data/diseases.txt', 'r') as dz_file:
    for line in dz_file:
        if line != '\n':
            line = line.strip('\n').split()
            if line[0] == 'DX':
                idx, name = parse_disease(line)
            elif line[0] == 'MX':
                idx, name, es, fq = parse_manifestation(line)
                # update manifestation use copy.copy() to create one for each dz?
            elif line[0] == 'LINK':
                idx, name = parse_link(line)

In [ ]:
# read in patient file
# build patient object -- (+MX, -MX)
# pull each possible dx_hyp from disease list (based on +MX)
# calculate score for each dx_hyp; save in ddx[]
# sort ddx[] in descending order
# choose optimal ddx or return list in order